In [1]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from LVQ.lvq import LVQ3
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
X_l = np.load("../data/processed/data.npy")
y_l = np.load("../data/processed/label.npy")
X_train, X_test, y_train, y_test = tts(X_l, y_l, test_size=0.20, random_state=15)
output_size = np.unique(y_l).shape[0]
input_size = X_train.shape[1]

In [3]:
# Learning 0.45, 0.35, 0.25
# Epoch: 5, 10, 15
# dMethod: "cosine", "euclidean"
# epsilon: 0.2, 0.3, 0.4
# Beta: 0.25, 0.3, 0.35
# decay: 0.05, 0.15, 0.25


In [4]:
param = {
    "lr": [0.25, 0.35, 0.45],
    "Epoch": [2, 3, 5],
    "epsilon": [0.25, 0.35],
    "beta": [0.25, 0.3],
    "decay": [0.15, 0.25],
    "dMethod": ["cosine", "euclidean"]
}

In [5]:
result = {}
tot_comb = 3 * 3 * 2 * 2 * 2 * 2

pbar = tqdm(desc="Grid searching", total=tot_comb)
Combination = 1
for dMethod in param["dMethod"]:
    for lr in param["lr"]:
        for epoch in param["Epoch"]:
            for eps in param["epsilon"]:
                for beta in param["beta"]:
                    for decay in param["decay"]:
                        # dMethod, lr, epoch, epsilon, beta, decay, akurasi
                        params = [dMethod, lr, epoch, eps, beta, decay]
                        grid = LVQ3(input_size=input_size, output_size=output_size, dMethod=dMethod, alpha=lr, epoch=epoch,
                                    epsilon=eps, beta=beta, decay=decay, random_state=12345)
                        grid.fit(X_train, y_train)
                        g_res = grid.predict(X_test)
                        params.append(accuracy_score(g_res, y_test))
                        t = "Combination_" + str(Combination)
                        result[t] = params
                        Combination += 1
                        pbar.update(1)

Grid searching: 100%|██████████| 144/144 [1:16:56<00:00, 38.98s/it]

In [9]:
from pprint import pprint
pprint(result)

{'Combination_1': ['cosine', 0.25, 2, 0.25, 0.25, 0.15, 0.4879778847760378],
 'Combination_10': ['cosine', 0.25, 3, 0.25, 0.25, 0.25, 0.5039567416498322],
 'Combination_100': ['euclidean',
                     0.35,
                     2,
                     0.25,
                     0.3,
                     0.25,
                     0.35060832054923524],
 'Combination_101': ['euclidean',
                     0.35,
                     2,
                     0.35,
                     0.25,
                     0.15,
                     0.3221592721418048],
 'Combination_102': ['euclidean',
                     0.35,
                     2,
                     0.35,
                     0.25,
                     0.25,
                     0.3221592721418048],
 'Combination_103': ['euclidean', 0.35, 2, 0.35, 0.3, 0.15, 0.3221592721418048],
 'Combination_104': ['euclidean', 0.35, 2, 0.35, 0.3, 0.25, 0.3221592721418048],
 'Combination_105': ['euclidean',
                     0.35

In [7]:
output_size = np.unique(y_l).shape[0]
input_size = X_train.shape[1]
sfs = LVQ3(input_size=input_size, output_size=output_size, random_state=201, epoch=3, alpha=0.45, beta=0.25, m=0.2,
           epsilon=0.25, dMethod="cosine")
sfs.fit(X_train, y_train)


In [8]:
res = sfs.predict(X_test)
print(classification_report(res, y_test, zero_division=1))


              precision    recall  f1-score   support

           0       0.65      0.77      0.71     19430
           1       0.23      0.44      0.30     11326
           2       0.78      0.48      0.59     35081

    accuracy                           0.56     65837
   macro avg       0.56      0.56      0.54     65837
weighted avg       0.65      0.56      0.58     65837
